<a href="https://colab.research.google.com/github/lazyboneOwO/PL-Repo/blob/main/Hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

SERVICE_ACCOUNT_KEY_FILE = 'service_account.json'
SPREADSHEET_NAME = 'hw2'
WORKSHEET_NAME = 'Sheet1'

def calculate_total_score(平时成绩, 期末成绩):
    try:
        平时成绩 = float(平时成绩)
        期末成绩 = float(期末成绩)
        total_score = (平时成绩 * 0.4) + (期末成绩 * 0.6)
        return round(total_score, 2)
    except ValueError:
        return "N/A"

def main():
    try:
        #認證服務帳戶
        scope = ['https://spreadsheets.google.com/feeds',
                 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_KEY_FILE, scope)
        client = gspread.authorize(creds)

        #開啟 Google 試算表和工作表
        spreadsheet = client.open(SPREADSHEET_NAME)
        worksheet = spreadsheet.worksheet(WORKSHEET_NAME)

        #讀取所有資料
        records = worksheet.get_all_records()
        headers = worksheet.row_values(1) # 獲取第一行的標題

        print(f"成功讀取 '{SPREADSHEET_NAME}' 的 '{WORKSHEET_NAME}' 工作表。")
        print("原始資料:")
        for record in records:
            print(record)

        # 找到總成績的欄位索引
        try:
            total_score_col_index = headers.index('總成績') + 1 # +1 因為 gspread 的欄位索引從 1 開始
        except ValueError:
            print("錯誤: 試算表中找不到 '總成績' 欄位。請檢查標題名稱。")
            return

        updates = []
        for i, record in enumerate(records):
            row_num = i + 2 # 資料從第二行開始，所以行號是索引 + 2
            平時成績 = record.get('平時成績')
            期末成績 = record.get('期末成績')

            if 平時成績 is not None and 期末成績 is not None:
                total_score = calculate_total_score(平時成績, 期末成績)
                if total_score != "N/A":
                    # 將更新資訊儲存起來，一次性寫入以提高效率
                    updates.append({
                        'range': f'{gspread.utils.rowcol_to_a1(row_num, total_score_col_index)}',
                        'values': [[str(total_score)]]
                    })
                    print(f"計算學生 {record.get('姓名', '未知')} 的總成績為: {total_score}")
                else:
                    print(f"學生 {record.get('姓名', '未知')} 的平時成績或期末成績無效，無法計算總成績。")
            else:
                print(f"學生 {record.get('姓名', '未知')} 缺少平時成績或期末成績。")

        # 4. 回寫總成績到試算表
        if updates:
            worksheet.batch_update(updates)
            print("總成績已成功回寫到 Google 試算表。")
        else:
            print("沒有需要更新的總成績。")

    except gspread.exceptions.SpreadsheetNotFound:
        print(f"錯誤: 找不到名稱為 '{SPREADSHEET_NAME}' 的試算表。請確認名稱是否正確。")
    except gspread.exceptions.WorksheetNotFound:
        print(f"錯誤: 找不到名稱為 '{WORKSHEET_NAME}' 的工作表。請確認名稱是否正確。")
    except Exception as e:
        print(f"發生錯誤: {e}")

if __name__ == "__main__":
    main()

成功讀取 'hw2' 的 'Sheet1' 工作表。
原始資料:
沒有需要更新的總成績。
